# SVM IY010_simulation_7 Analysis: Varying 1 stat, fixing 2 stats

In [2]:
from __future__ import annotations
import numpy as np
import pandas as pd
import glob
from pathlib import Path
from typing import Dict, Optional
import sys
import os
import re
from collections import defaultdict
import torch
import time

# Add src directory to Python path
sys.path.append(str(Path.cwd().parent.parent.parent / "src"))

# Import custom modules
from classifiers.svm_classifier import svm_classifier, grid_search_svm
from models.TF_transformer import TFTransformer, ModelCfg
from utils.data_processing import add_binary_labels, add_nearest_neighbour_labels
from utils.standardise_time_series import standardise_time_series
from utils.shuffle_time_series import shuffle_time_series

# Import sklearn modules for SVM
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Visualise results when we vary 1 stat target at a time

Changing ``t_ac_target``:

In [ ]:
##### Set up directory paths for data loading ######
BASE_DIR = Path.cwd().parent
OUT_DIR = BASE_DIR
SYNTHETIC_DIR = BASE_DIR / "data_7_mu_cv_fixed" 
RESULTS_CSV = "IY010_simulation_parameters_7_mu_cv_fixed.csv"
results_csv_path = BASE_DIR / RESULTS_CSV
results = pd.read_csv(results_csv_path)
results = results[results["success"]].dropna(
    subset=["mu_observed", "cv_observed", "t_ac_observed"]
)

label_column = 't_ac_target' 
labelled_results = add_binary_labels(results, label_column)
# labelled_results = add_nearest_neighbour_labels(results, positive_on=label_column)
##### Set up directory paths for data loading ######

##### Use standardise_time_series utility function ######
# Collect all DataFrames and their labels
data_frames = []
labels = []

for i in range(len(results)):
    trajectory_filename = results["trajectory_filename"].values[i]
    DATA_CSV = SYNTHETIC_DIR / trajectory_filename
    data = pd.read_csv(DATA_CSV)
    data_frames.append(data)
    
    # Get the corresponding label from the results csv
    label_value = labelled_results[labelled_results['trajectory_filename'] == trajectory_filename]['label'].iloc[0]
    labels.append(label_value)

# Use the utility function to standardize
labelled_data = standardise_time_series(data_frames, labels=labels, prefix="t_")

print(f"📏 Standardized dataset:")
print(f"   NaN values: {labelled_data.isnull().sum().sum()}")
##### Use standardise_time_series utility function ######

# =========================================================
# Prepare Features and Labels for SVM
# =========================================================
df = labelled_data.copy()
# Extract labels
y = df["label"].values

# Extract features (all columns except 'label')
X = df.drop(columns=["label"]).values

print(f"Data preparation for SVM:")
print(f"  Feature matrix shape: {X.shape}")
print(f"  Labels shape: {y.shape}")
print(f"  Number of classes: {len(np.unique(y))}")
print(f"  Class distribution: {np.bincount(y)}")
print(f"  Memory usage: {X.nbytes / 1024**2:.2f} MB")

# Check for any NaN or infinite values
if np.any(np.isnan(X)):
    print("⚠️  Warning: NaN values detected in features")
if np.any(np.isinf(X)):
    print("⚠️  Warning: Infinite values detected in features")
    
print("✅ Data ready for SVM classification!")

# SVM Parameters (using defaults from svm_classifier function)
SVM_C = 1.0           # Regularization parameter
SVM_GAMMA = 'scale'   # Kernel coefficient 
SVM_KERNEL = 'rbf'    # Kernel type

# Train/test split ratio
TEST_SPLIT = 0.2
RANDOM_STATE = 42

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=TEST_SPLIT, 
    random_state=RANDOM_STATE,
    stratify=y  # Ensure balanced split across classes
)

# Record training time
start_time = time.time()

# Train SVM using the imported svm_classifier function
svm_accuracy = svm_classifier(
    X_train, X_test, y_train, y_test,
    svm_C=SVM_C,
    svm_gamma=SVM_GAMMA, 
    svm_kernel=SVM_KERNEL,
    print_classification_report=True,
    print_confusion_matrix=True,
)

training_time = time.time() - start_time
print(f"⏱️  SVM ({SVM_KERNEL}) training and evaluation time: {training_time:.2f} seconds")

# =========================================================
# Experiment: Does Temporal Order Matter for Classification?
# =========================================================
print("\n🔀 Experiment: Shuffling Time Series Data to Test Temporal Order Impact")
# Use the utility function to create shuffled data
df_shuffled = shuffle_time_series(
    df, 
    preserve_columns=['label'], 
    random_state=RANDOM_STATE
)

# Extract features and labels from shuffled data
y_shuffled = df_shuffled["label"].values
X_shuffled = df_shuffled.drop(columns=["label"]).values

# Split the shuffled data
X_train_shuffled, X_test_shuffled, y_train_shuffled, y_test_shuffled = train_test_split(
    X_shuffled, 
    y_shuffled, 
    test_size=TEST_SPLIT,
    random_state=RANDOM_STATE,
    stratify=y_shuffled
)

# Record training time
start_time = time.time()

# Train SVM on shuffled data
svm_accuracy_shuffled = svm_classifier(
    X_train_shuffled, X_test_shuffled, y_train_shuffled, y_test_shuffled,
    svm_C=SVM_C,
    svm_gamma=SVM_GAMMA, 
    svm_kernel=SVM_KERNEL,
    print_classification_report=True,
    print_confusion_matrix=True,
)
training_time = time.time() - start_time
print(f"⏱️  SVM ({SVM_KERNEL}) training and evaluation time: {training_time:.2f} seconds")

📏 Standardized dataset:
   NaN values: 0
Data preparation for SVM:
  Feature matrix shape: (9400, 174)
  Labels shape: (9400,)
  Number of classes: 2
  Class distribution: [4600 4800]
  Memory usage: 12.48 MB
✅ Data ready for SVM classification!
=== SVM (RBF Kernel) Classification Accuracy: 0.83 ===

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.72      0.81       920
           1       0.78      0.94      0.85       960

    accuracy                           0.83      1880
   macro avg       0.85      0.83      0.83      1880
weighted avg       0.85      0.83      0.83      1880


Confusion Matrix:
[[665 255]
 [ 58 902]]
⏱️  SVM (rbf) training and evaluation time: 4.13 seconds

🔀 Experiment: Shuffling Time Series Data to Test Temporal Order Impact
=== SVM (RBF Kernel) Classification Accuracy: 0.70 ===

Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.63      0.67

Changing ``cv_target``:

In [ ]:
##### Set up directory paths for data loading ######
BASE_DIR = Path.cwd().parent
OUT_DIR = BASE_DIR
SYNTHETIC_DIR = BASE_DIR / "data_7_mu_t_ac_fixed" 
RESULTS_CSV = "IY010_simulation_parameters_7_mu_t_ac_fixed.csv"
results_csv_path = BASE_DIR / RESULTS_CSV
results = pd.read_csv(results_csv_path)
results = results[results["success"]].dropna(
    subset=["mu_observed", "cv_observed", "t_ac_observed"]
)

label_column = "cv_target" 
labelled_results = add_binary_labels(results, label_column)
# labelled_results = add_nearest_neighbour_labels(results, positive_on=label_column)
##### Set up directory paths for data loading ######

##### Use standardise_time_series utility function ######
# Collect all DataFrames and their labels
data_frames = []
labels = []

for i in range(len(results)):
    trajectory_filename = results["trajectory_filename"].values[i]
    DATA_CSV = SYNTHETIC_DIR / trajectory_filename
    data = pd.read_csv(DATA_CSV)
    data_frames.append(data)
    
    # Get the corresponding label from the results csv
    label_value = labelled_results[labelled_results['trajectory_filename'] == trajectory_filename]['label'].iloc[0]
    labels.append(label_value)

# Use the utility function to standardize
labelled_data = standardise_time_series(data_frames, labels=labels, prefix="t_")

print(f"📏 Standardized dataset:")
print(f"   NaN values: {labelled_data.isnull().sum().sum()}")
##### Use standardise_time_series utility function ######

# =========================================================
# Prepare Features and Labels for SVM
# =========================================================
df = labelled_data.copy()
# Extract labels
y = df["label"].values

# Extract features (all columns except 'label')
X = df.drop(columns=["label"]).values

print(f"Data preparation for SVM:")
print(f"  Feature matrix shape: {X.shape}")
print(f"  Labels shape: {y.shape}")
print(f"  Number of classes: {len(np.unique(y))}")
print(f"  Class distribution: {np.bincount(y)}")
print(f"  Memory usage: {X.nbytes / 1024**2:.2f} MB")

# Check for any NaN or infinite values
if np.any(np.isnan(X)):
    print("⚠️  Warning: NaN values detected in features")
if np.any(np.isinf(X)):
    print("⚠️  Warning: Infinite values detected in features")
    
print("✅ Data ready for SVM classification!")

# SVM Parameters (using defaults from svm_classifier function)
SVM_C = 1.0           # Regularization parameter
SVM_GAMMA = 'scale'   # Kernel coefficient 
SVM_KERNEL = 'rbf'    # Kernel type

# Train/test split ratio
TEST_SPLIT = 0.2
RANDOM_STATE = 42

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=TEST_SPLIT, 
    random_state=RANDOM_STATE,
    stratify=y  # Ensure balanced split across classes
)

# Record training time
start_time = time.time()

# Train SVM using the imported svm_classifier function
svm_accuracy = svm_classifier(
    X_train, X_test, y_train, y_test,
    svm_C=SVM_C,
    svm_gamma=SVM_GAMMA, 
    svm_kernel=SVM_KERNEL,
    print_classification_report=True,
    print_confusion_matrix=True,
)

training_time = time.time() - start_time
print(f"⏱️  SVM ({SVM_KERNEL}) training and evaluation time: {training_time:.2f} seconds")

# =========================================================
# Experiment: Does Temporal Order Matter for Classification?
# =========================================================
print("\n🔀 Experiment: Shuffling Time Series Data to Test Temporal Order Impact")
# Use the utility function to create shuffled data
df_shuffled = shuffle_time_series(
    df, 
    preserve_columns=['label'], 
    random_state=RANDOM_STATE
)

# Extract features and labels from shuffled data
y_shuffled = df_shuffled["label"].values
X_shuffled = df_shuffled.drop(columns=["label"]).values

# Split the shuffled data
X_train_shuffled, X_test_shuffled, y_train_shuffled, y_test_shuffled = train_test_split(
    X_shuffled, 
    y_shuffled, 
    test_size=TEST_SPLIT,
    random_state=RANDOM_STATE,
    stratify=y_shuffled
)

# Record training time
start_time = time.time()

# Train SVM on shuffled data
svm_accuracy_shuffled = svm_classifier(
    X_train_shuffled, X_test_shuffled, y_train_shuffled, y_test_shuffled,
    svm_C=SVM_C,
    svm_gamma=SVM_GAMMA, 
    svm_kernel=SVM_KERNEL,
    print_classification_report=True,
    print_confusion_matrix=True,
)
training_time = time.time() - start_time
print(f"⏱️  SVM ({SVM_KERNEL}) training and evaluation time: {training_time:.2f} seconds")

📏 Standardized dataset:
   NaN values: 0
Data preparation for SVM:
  Feature matrix shape: (9600, 194)
  Labels shape: (9600,)
  Number of classes: 2
  Class distribution: [4800 4800]
  Memory usage: 14.21 MB
✅ Data ready for SVM classification!
=== SVM (RBF Kernel) Classification Accuracy: 0.92 ===

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.91      0.92       960
           1       0.91      0.93      0.92       960

    accuracy                           0.92      1920
   macro avg       0.92      0.92      0.92      1920
weighted avg       0.92      0.92      0.92      1920


Confusion Matrix:
[[877  83]
 [ 72 888]]
⏱️  SVM (rbf) training and evaluation time: 2.05 seconds

🔀 Experiment: Shuffling Time Series Data to Test Temporal Order Impact
=== SVM (RBF Kernel) Classification Accuracy: 0.91 ===

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.90      0.91

Changing ``mu_target``:

In [ ]:
##### Set up directory paths for data loading ######
BASE_DIR = Path.cwd().parent
OUT_DIR = BASE_DIR
SYNTHETIC_DIR = BASE_DIR / "data_7_cv_t_ac_fixed" 
RESULTS_CSV = "IY010_simulation_parameters_7_cv_t_ac_fixed.csv"
results_csv_path = BASE_DIR / RESULTS_CSV
results = pd.read_csv(results_csv_path)
results = results[results["success"]].dropna(
    subset=["mu_observed", "cv_observed", "t_ac_observed"]
)

label_column = "mu_target" 
labelled_results = add_binary_labels(results, label_column)
# labelled_results = add_nearest_neighbour_labels(results, positive_on=label_column)
##### Set up directory paths for data loading ######

##### Use standardise_time_series utility function ######
# Collect all DataFrames and their labels
data_frames = []
labels = []

for i in range(len(results)):
    trajectory_filename = results["trajectory_filename"].values[i]
    DATA_CSV = SYNTHETIC_DIR / trajectory_filename
    data = pd.read_csv(DATA_CSV)
    data_frames.append(data)
    
    # Get the corresponding label from the results csv
    label_value = labelled_results[labelled_results['trajectory_filename'] == trajectory_filename]['label'].iloc[0]
    labels.append(label_value)

# Use the utility function to standardize
labelled_data = standardise_time_series(data_frames, labels=labels, prefix="t_")

print(f"📏 Standardized dataset:")
print(f"   NaN values: {labelled_data.isnull().sum().sum()}")
##### Use standardise_time_series utility function ######

# =========================================================
# Prepare Features and Labels for SVM
# =========================================================
df = labelled_data.copy()
# Extract labels
y = df["label"].values

# Extract features (all columns except 'label')
X = df.drop(columns=["label"]).values

print(f"Data preparation for SVM:")
print(f"  Feature matrix shape: {X.shape}")
print(f"  Labels shape: {y.shape}")
print(f"  Number of classes: {len(np.unique(y))}")
print(f"  Class distribution: {np.bincount(y)}")
print(f"  Memory usage: {X.nbytes / 1024**2:.2f} MB")

# Check for any NaN or infinite values
if np.any(np.isnan(X)):
    print("⚠️  Warning: NaN values detected in features")
if np.any(np.isinf(X)):
    print("⚠️  Warning: Infinite values detected in features")
    
print("✅ Data ready for SVM classification!")

# SVM Parameters (using defaults from svm_classifier function)
SVM_C = 1.0           # Regularization parameter
SVM_GAMMA = 'scale'   # Kernel coefficient 
SVM_KERNEL = 'rbf'    # Kernel type

# Train/test split ratio
TEST_SPLIT = 0.2
RANDOM_STATE = 42

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=TEST_SPLIT, 
    random_state=RANDOM_STATE,
    stratify=y  # Ensure balanced split across classes
)

# Record training time
start_time = time.time()

# Train SVM using the imported svm_classifier function
svm_accuracy = svm_classifier(
    X_train, X_test, y_train, y_test,
    svm_C=SVM_C,
    svm_gamma=SVM_GAMMA, 
    svm_kernel=SVM_KERNEL,
    print_classification_report=True,
    print_confusion_matrix=True,
)

training_time = time.time() - start_time
print(f"⏱️  SVM ({SVM_KERNEL}) training and evaluation time: {training_time:.2f} seconds")

# =========================================================
# Experiment: Does Temporal Order Matter for Classification?
# =========================================================
print("\n🔀 Experiment: Shuffling Time Series Data to Test Temporal Order Impact")
# Use the utility function to create shuffled data
df_shuffled = shuffle_time_series(
    df, 
    preserve_columns=['label'], 
    random_state=RANDOM_STATE
)

# Extract features and labels from shuffled data
y_shuffled = df_shuffled["label"].values
X_shuffled = df_shuffled.drop(columns=["label"]).values

# Split the shuffled data
X_train_shuffled, X_test_shuffled, y_train_shuffled, y_test_shuffled = train_test_split(
    X_shuffled, 
    y_shuffled, 
    test_size=TEST_SPLIT,
    random_state=RANDOM_STATE,
    stratify=y_shuffled
)

# Record training time
start_time = time.time()

# Train SVM on shuffled data
svm_accuracy_shuffled = svm_classifier(
    X_train_shuffled, X_test_shuffled, y_train_shuffled, y_test_shuffled,
    svm_C=SVM_C,
    svm_gamma=SVM_GAMMA, 
    svm_kernel=SVM_KERNEL,
    print_classification_report=True,
    print_confusion_matrix=True,
)
training_time = time.time() - start_time
print(f"⏱️  SVM ({SVM_KERNEL}) training and evaluation time: {training_time:.2f} seconds")